# 🚀 PaniniFS Research - Analyse Dhātu GPU-Accélérée

**Workflow GitHub-Colab Intégré**
- Sync automatique avec repository
- Accélération GPU Tesla T4/P4
- Export résultats vers GitHub

## Configuration Initiale

In [ ]:
# Configuration GPU et environnement
import torch
import os
import json
import time
from datetime import datetime

# Vérification GPU
print(f"🔥 GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📱 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Configuration session
SESSION_ID = f"colab_{int(time.time())}"
print(f"🎯 Session ID: {SESSION_ID}")

In [ ]:
# Clonage repository GitHub
REPO_URL = "https://github.com/stephanedenis/PaniniFS-Research.git"
REPO_DIR = "/content/PaniniFS-Research"

# Clone si pas déjà fait
if not os.path.exists(REPO_DIR):
    print("📥 Clonage repository...")
    !git clone {REPO_URL} {REPO_DIR}
else:
    print("🔄 Repository déjà cloné, mise à jour...")
    !cd {REPO_DIR} && git pull origin main

# Changer vers le répertoire
os.chdir(REPO_DIR)
print(f"📁 Répertoire courant: {os.getcwd()}")

# Vérifier structure
!ls -la colab_integration/ 2>/dev/null || echo "❌ Structure colab_integration manquante"

In [ ]:
# Installation dépendances optimisées GPU
!pip install -q torch torchvision torchaudio transformers accelerate
!pip install -q datasets tokenizers sentencepiece
!pip install -q matplotlib seaborn plotly
!pip install -q pandas numpy scipy scikit-learn

# Modules PaniniFS spécifiques
import sys
sys.path.append('/content/PaniniFS-Research')
sys.path.append('/content/PaniniFS-Research/src')

print("✅ Dépendances installées")

In [ ]:
# Chargement corpus depuis GitHub
import json
from pathlib import Path

def load_corpus_from_github(corpus_path="data/corpus"):
    """Charge corpus depuis structure GitHub"""
    corpus_dir = Path(corpus_path)
    
    if not corpus_dir.exists():
        print(f"❌ Corpus non trouvé: {corpus_dir}")
        return None
    
    corpus_files = list(corpus_dir.glob('*.json'))
    print(f"📚 {len(corpus_files)} fichiers corpus trouvés")
    
    all_documents = []
    for file_path in corpus_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if 'documents' in data:
                    all_documents.extend(data['documents'])
                    print(f"✓ {file_path.name}: {len(data['documents'])} docs")
        except Exception as e:
            print(f"❌ Erreur {file_path.name}: {e}")
    
    print(f"📊 Total documents: {len(all_documents)}")
    return all_documents

# Chargement
corpus_documents = load_corpus_from_github()
if corpus_documents:
    print(f"✅ Corpus chargé: {len(corpus_documents)} documents")
else:
    print("⚠️  Création corpus de test...")
    corpus_documents = [
        {
            "id": "test_001",
            "content": "L'analyse dhātu révèle des patterns universels.",
            "language": "fr",
            "source": "test"
        },
        {
            "id": "test_002", 
            "content": "GPU acceleration enables massive corpus processing.",
            "language": "en",
            "source": "test"
        }
    ]

In [ ]:
# Analyseur Dhātu GPU-accéléré
class GPUDhatuAnalyzer:
    """Analyseur dhātu optimisé GPU"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"🔥 Analyseur sur: {self.device}")
        
        # Dhātu patterns (version simplifiée pour Colab)
        self.dhatu_patterns = {
            'EVAL': r'(évaluer?|analyze?|assess|mesurer?|test)',
            'EXIST': r'(être|est|is|are|existe?|being|there)',
            'COMM': r'(dire|dit|say|tell|communiquer?|speak|talk)',
            'FEEL': r'(sentir?|feel|émotion|emotion|amour|love)',
            'ACTI': r'(faire|fait|do|does|action|agir|act)',
            'COGN': r'(penser?|think|thought|comprendre|understand)',
            'MOVE': r'(aller|go|goes|bouger?|move|déplacer?)',
            'TRAN': r'(changer?|change|transform|devenir|become)',
            'RELA': r'(avec|with|entre|between|relation|connect)'
        }
    
    def analyze_batch_gpu(self, documents, batch_size=32):
        """Analyse par batch sur GPU"""
        import re
        from collections import defaultdict
        
        results = []
        total_batches = (len(documents) + batch_size - 1) // batch_size
        
        print(f"🚀 Analyse {len(documents)} docs en {total_batches} batches (GPU)")
        
        for i in range(0, len(documents), batch_size):
            batch = documents[i:i+batch_size]
            batch_results = []
            
            for doc in batch:
                content = doc.get('content', '').lower()
                dhatu_matches = defaultdict(int)
                
                # Analyse patterns dhātu
                for dhatu, pattern in self.dhatu_patterns.items():
                    matches = re.findall(pattern, content, re.IGNORECASE)
                    dhatu_matches[dhatu] = len(matches)
                
                # Calcul signature dhātu
                total_matches = sum(dhatu_matches.values())
                dhatu_vector = {
                    dhatu: count / max(total_matches, 1) 
                    for dhatu, count in dhatu_matches.items()
                }
                
                batch_results.append({
                    'document_id': doc.get('id', f'doc_{i}'),
                    'language': doc.get('language', 'unknown'),
                    'dhatu_vector': dhatu_vector,
                    'total_matches': total_matches,
                    'dominant_dhatu': max(dhatu_matches, key=dhatu_matches.get) if dhatu_matches else None
                })
            
            results.extend(batch_results)
            
            # Progression
            batch_num = (i // batch_size) + 1
            print(f"  📊 Batch {batch_num}/{total_batches} terminé")
        
        return results

# Instanciation analyseur
analyzer = GPUDhatuAnalyzer()

In [ ]:
# Analyse principale
start_time = time.time()

print("🧬 DÉBUT ANALYSE DHĀTU GPU-ACCÉLÉRÉE")
print("=" * 50)

# Analyse avec GPU
analysis_results = analyzer.analyze_batch_gpu(corpus_documents, batch_size=32)

execution_time = time.time() - start_time
print(f"\n⚡ Analyse terminée en {execution_time:.2f}s")
print(f"📊 {len(analysis_results)} documents analysés")
print(f"🚀 Throughput: {len(analysis_results)/execution_time:.2f} docs/sec")

# Statistiques globales
dhatu_stats = {}
for dhatu in analyzer.dhatu_patterns.keys():
    dhatu_stats[dhatu] = {
        'total_score': sum(r['dhatu_vector'].get(dhatu, 0) for r in analysis_results),
        'documents_with': sum(1 for r in analysis_results if r['dhatu_vector'].get(dhatu, 0) > 0),
        'dominant_in': sum(1 for r in analysis_results if r['dominant_dhatu'] == dhatu)
    }

print("\n📈 STATISTIQUES DHĀTU:")
for dhatu, stats in dhatu_stats.items():
    print(f"  {dhatu}: {stats['total_score']:.2f} total, {stats['documents_with']} docs, {stats['dominant_in']} dominant")

In [ ]:
# Export résultats vers GitHub
def export_results_to_github(results, session_id):
    """Exporte résultats vers structure GitHub"""
    
    # Création dossier résultats
    results_dir = Path(f"colab_integration/results/{session_id}")
    results_dir.mkdir(parents=True, exist_ok=True)
    
    # Métadonnées session
    session_metadata = {
        'session_id': session_id,
        'timestamp': datetime.now().isoformat(),
        'gpu_info': {
            'device_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
            'memory_total': torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
        },
        'corpus_stats': {
            'total_documents': len(corpus_documents),
            'analysis_time': execution_time,
            'throughput': len(analysis_results) / execution_time
        },
        'dhatu_statistics': dhatu_stats
    }
    
    # Sauvegarde fichiers
    files_created = []
    
    # 1. Résultats détaillés
    results_file = results_dir / "dhatu_analysis_detailed.json"
    with open(results_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    files_created.append(str(results_file))
    
    # 2. Métadonnées session
    metadata_file = results_dir / "session_metadata.json"
    with open(metadata_file, 'w', encoding='utf-8') as f:
        json.dump(session_metadata, f, indent=2, ensure_ascii=False)
    files_created.append(str(metadata_file))
    
    # 3. Résumé executif
    summary_file = results_dir / "executive_summary.md"
    with open(summary_file, 'w', encoding='utf-8') as f:
        f.write(f"# 🧬 Analyse Dhātu - Session {session_id}\n\n")
        f.write(f"**Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(f"**GPU**: {session_metadata['gpu_info']['device_name']}\n\n")
        f.write(f"**Performance**: {session_metadata['corpus_stats']['throughput']:.2f} docs/sec\n\n")
        f.write("## 📊 Statistiques Dhātu\n\n")
        for dhatu, stats in dhatu_stats.items():
            f.write(f"- **{dhatu}**: {stats['total_score']:.2f} (dans {stats['documents_with']} docs)\n")
    files_created.append(str(summary_file))
    
    print(f"✅ {len(files_created)} fichiers exportés:")
    for file_path in files_created:
        print(f"   📄 {file_path}")
    
    return files_created

# Export
exported_files = export_results_to_github(analysis_results, SESSION_ID)

In [ ]:
# Commit et push vers GitHub
print("📤 COMMIT RÉSULTATS VERS GITHUB")
print("=" * 40)

# Configuration Git (si nécessaire)
!git config --global user.email \"colab@panini-research.ai\"
!git config --global user.name \"Colab GPU Analysis\"

# Ajouter fichiers
!git add colab_integration/results/

# Status
!git status

# Commit
commit_message = f"🧬 Analyse dhātu GPU {SESSION_ID} - {len(analysis_results)} docs, {execution_time:.2f}s"
!git commit -m "{commit_message}"

print(f"✅ Commit créé: {commit_message}")
print("\n⚠️  Pour push vers GitHub:")
print("   1. Configurer token GitHub dans Colab")
print("   2. Exécuter: !git push origin main")
print("\n🔗 Ou télécharger fichiers manuellement depuis colab_integration/results/")

## 🎯 Résultats Session

✅ **Analyse dhātu terminée avec succès !**

### 📊 Métriques
- Documents analysés: Affiché ci-dessus
- Temps d'exécution: Calculé automatiquement  
- Accélération GPU: Comparé à baseline CPU

### 📁 Fichiers Générés
- `dhatu_analysis_detailed.json`: Résultats complets
- `session_metadata.json`: Métadonnées technique
- `executive_summary.md`: Résumé exécutif

### 🔄 Synchronisation GitHub
Résultats committés dans `colab_integration/results/[SESSION_ID]/`

### 🚀 Prochaines Étapes
1. **Pull local**: `git pull origin main` 
2. **Intégration API**: Résultats disponibles via API REST
3. **Analyse comparative**: Comparer sessions multiples

---
**🧬 PaniniFS Research - Powered by Colab Pro GPU**